In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

In [2]:
%nuclio cmd pip install textblob
%nuclio env TO_LANG=fr

%nuclio: setting 'TO_LANG' environment variable


In [11]:
from textblob import TextBlob
import os

def handler(context, event):
    context.logger.info('This is an NLP example! ')

    # process and correct the text
    blob = TextBlob(str(event.body.decode('utf-8')))
    corrected = blob.correct()

    # debug print the text before and after correction
    context.logger.info_with("Corrected text", corrected=str(corrected), orig=str(blob))

    # calculate sentiments
    context.logger.info_with("Sentiment",
                             polarity=str(corrected.sentiment.polarity),
                             subjectivity=str(corrected.sentiment.subjectivity))

    # read target language from environment and return translated text
    lang = os.getenv('TO_LANG','fr')
    return str(corrected.translate(to=lang))

In [16]:
# nuclio: ignore
event = nuclio.Event(body=b'good morninng')
handler(context, event)

Python> 2019-03-14 11:33:43,830 [info] This is an NLP example! 
Python> 2019-03-14 11:33:43,831 [info] Corrected text: {'corrected': 'good evening', 'orig': 'good evening'}
Python> 2019-03-14 11:33:43,833 [info] Sentiment: {'polarity': '0.7', 'subjectivity': '0.6000000000000001'}


'Bonsoir'

In [14]:
%nuclio deploy -n nlp -p ai -c

%nuclio: ['deploy', '-n', 'nlp', '-p', 'ai', '-c', '/User/nlp-example.ipynb']
%nuclio: [nuclio.deploy] 2019-03-14 11:32:56,340 (info) Building processor image
%nuclio: [nuclio.deploy] 2019-03-14 11:32:58,391 (info) Pushing image
%nuclio: [nuclio.deploy] 2019-03-14 11:32:58,391 (info) Build complete
%nuclio: [nuclio.deploy] 2019-03-14 11:33:02,434 (info) Function deploy complete
%nuclio: [nuclio.deploy] 2019-03-14 11:33:02,442 done updating nlp, function address: 3.122.204.208:32182
%nuclio: function deployed


In [15]:
!curl -X POST -d "good evening" 3.122.204.208:32182

bonne soir��e '